In [25]:
import numpy as np
import fnmatch
import xarray as xr
import os
from collections import defaultdict
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
from xmip.postprocessing import combine_datasets,_concat_sorted_time
from sklearn.decomposition import PCA
import gcsfs
from cmip_ds_dict_operations import select_period, pr_flux_to_m, drop_duplicate_timesteps, drop_coords, drop_incomplete
fs = gcsfs.GCSFileSystem() # equivalent to fsspec.fs('gs')

grid_size = 9 #n by n degree grid around each TG
cmip6_resolution = 1.5

overwrite_existing = False
num_grid_cells = int(grid_size/cmip6_resolution)

start_year=1970
end_year=2100

mlr_coefs = xr.open_dataset('/home/jovyan/CMIP6cex/cmip6_processing/gssr_mlr_coefs_1p5_9deg_gesla2.nc') #load MLR coefficients at TGs
era5_pcs = xr.open_dataset('/home/jovyan/CMIP6cex/cmip6_processing/era5_pca_components_1p5_9deg_gesla2.nc')

Loop over timeseries of `psl` & `sfcWind` at common 1.5 by 1.5 degree grid and open them:

In [42]:
var1 = 'pr'
var2 = 'sfcWind'

var1_dir = 'leap-persistent/timh37/CMIP6/datasets_eu_gesla2_tgs/'+var1
var2_dir = 'leap-persistent/timh37/CMIP6/datasets_eu_gesla2_tgs/'+var2

ddict = defaultdict(dict)

for m in tqdm(fs.ls(var1_dir)):
    for f in tqdm(fs.ls(m)):
        ds = xr.open_dataset('gs://'+f,engine='zarr',chunks={},use_cftime=True)
        ddict[f.split('/')[-1]] = ds



'''
for top, dirs, files in os.walk(in_dir):
        for file in files:
            ds_fns.append(os.path.join(top, file))
#output_dir = 'leap-persistent/timh37/CMIP6/timeseries_eu_gesla2_tgs/'

var1_models = [k.split('/')[-1] for k in fs.ls(var1_dir) if k.startswith('.')==False]
var2_models = [k.split('/')[-1] for k in fs.ls(var2_dir) if k.startswith('.')==False]

models = [k for k in var1_models if k in var2_models]
#models = ['EC-Earth3']
ddict = defaultdict(dict)

'''


'''
for s,source_id in tqdm(enumerate(models)):
    for f,file in enumerate(fs.ls(os.path.join(var1_dir,source_id))):
        
        try: #why is this taking almost 2s per instance? chunksizes look good to me ~100mb..
            var1_var2_ds = xr.open_mfdataset(('gs://'+file,'gs://'+file.replace(var1,var2)),engine='zarr',use_cftime=True)
        except:
            continue
        
        k = file.split('/')[-1]
        ddict[k] = var1_var2_ds.sel(time=slice(str(start_year), str(end_year))
'''

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:724: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:557: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()


  0%|          | 0/120 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:724: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:557: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:724: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:557:

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/154 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
ddict_ssp245 = defaultdict(dict)
ddict_ssp585 = defaultdict(dict)
for k,v in ddict.items():
    if 'ssp585' in k:
        ddict_ssp585[k]=v
    elif 'ssp245' in k:
        ddict_ssp245[k]=v
    elif 'hist' in k:
        if 
        ddict_ssp245[k]=v
        ddict_ssp585[k]=v

In [37]:
'.'.join('CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r10i1p1f1.day.psl.gn'.split('.')[2::]).replace('historical','ssp245')

'ACCESS-CM2.ssp245.r10i1p1f1.day.psl.gn'

In [21]:
hist_ssp245 = combine_datasets(ddict_ssp245,_concat_sorted_time,match_attrs =['source_id', 'grid_label','table_id','variant_label','variable_id'],combine_func_kwargs={'join':'inner','coords':'minimal'})
hist_ssp585 = combine_datasets(ddict_ssp245,_concat_sorted_time,match_attrs =['source_id', 'grid_label','table_id','variant_label','variable_id'],combine_func_kwargs={'join':'inner','coords':'minimal'})

/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/postprocessing.py:157: UserWarning: CMIP.EC-Earth-Consortium.EC-Earth3.historical.r24i1p1f1.day.gr.none.psl failed to combine with :The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/postprocessing.py:157: UserWarning: CMIP.EC-Earth-Consortium.EC-Earth3.historical.r24i1p1f1.day.gr.none.psl failed to combine with :The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single

In [28]:
hist_ssp245_ = defaultdict(dict) #probably a better way to do this, but there are approx. 1 files for which the time units are inconsistent between historical and ssp
for k,v in hist_ssp245.items():
    if v.time[-1].values.dtype != v.time[0].values.dtype:
        print('dropping ' + k +' due to inconsistent timestamps in historical and ssp runs')
        continue
    else:
        hist_ssp245_[k] = v
            
hist_ssp245_ = drop_duplicate_timesteps(hist_ssp245_) #remove overlap between historical and ssp experiments which sometimes exists
hist_ssp245_complete = drop_incomplete(hist_ssp245_)
hist_ssp245_complete_ = defaultdict(dict)
for k,v in hist_ssp245_complete.items():
    if v.time.dt.year>=2099:
        hist_ssp245_complete_[k] = v
    

Dropping duplicate timesteps for:CESM2.gn.day.r11i1p1f1.psl
Dropping duplicate timesteps for:CESM2.gn.day.r4i1p1f1.psl
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r1i1p1f1.psl
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r2i1p1f1.psl
Dropping duplicate timesteps for:CESM2-WACCM.gn.day.r3i1p1f1.psl
Dropping duplicate timesteps for:EC-Earth3-Veg.gr.day.r5i1p1f1.psl


In [39]:
hist_ssp245_complete_ = defaultdict(dict)
for k,v in hist_ssp245_complete.items():
    if v.time.dt.year[-1]>=2099:
        hist_ssp245_complete_[k] = v

In [41]:
hist_ssp245_complete_

defaultdict(dict,
            {'ACCESS-CM2.gn.day.r1i1p1f1.psl': <xarray.Dataset>
             Dimensions:    (latitude: 27, longitude: 35, member_id: 1, time: 91676)
             Coordinates:
               * latitude   (latitude) float64 70.0 68.5 67.0 65.5 ... 35.5 34.0 32.5 31.0
               * longitude  (longitude) float64 -30.0 -28.5 -27.0 -25.5 ... 18.0 19.5 21.0
               * member_id  (member_id) object 'r1i1p1f1'
               * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2100-12-31T12:00:00
             Data variables:
                 psl        (member_id, time, latitude, longitude) float32 dask.array<chunksize=(1, 40000, 27, 35), meta=np.ndarray>
             Attributes: (12/66)
                 Conventions:                      CF-1.7 CMIP-6.2
                 activity_id:                      CMIP
                 branch_method:                    standard
                 branch_time_in_child:             0.0
                 branch_time_in_parent: 

In [29]:
hist_ssp585_ = defaultdict(dict) #probably a better way to do this, but there are approx. 1 files for which the time units are inconsistent between historical and ssp
for k,v in hist_ssp245.items():
    if v.time[-1].values.dtype != v.time[0].values.dtype:
        print('dropping ' + k +' due to inconsistent timestamps in historical and ssp runs')
        continue
    else:
        hist_ssp585_[k] = v
            
hist_ssp585_ = drop_duplicate_timesteps(hist_ssp585_) #remove overlap between historical and ssp experiments which sometimes exists
hist_ssp585_complete = drop_incomplete(hist_ssp585_)

280

In [5]:
#generate lon/lat indices around each TG
ds0 = ddict[list(ddict)[0]]

lat_ranges = np.zeros((len(mlr_coefs.tg),int(grid_size/cmip6_resolution))) #initialize
lon_ranges = np.zeros((len(mlr_coefs.tg),int(grid_size/cmip6_resolution)))

for t,tg in enumerate(mlr_coefs.tg.values): #grids around TGs
    lat_ranges[t,:] = ds0.latitude[((ds0.latitude>=(mlr_coefs.sel(tg=tg).lat-grid_size/2)) & (ds0.latitude<=(mlr_coefs.sel(tg=tg).lat+grid_size/2)))][0:int(grid_size/cmip6_resolution)]
    lon_ranges[t,:] = ds0.longitude[((ds0.longitude>=(mlr_coefs.sel(tg=tg).lon-grid_size/2)) & (ds0.longitude<=(mlr_coefs.sel(tg=tg).lon+grid_size/2)))][0:int(grid_size/cmip6_resolution)]

lons_da = xr.DataArray(lon_ranges,dims=['tg','lon_around_tg'],coords={'tg':mlr_coefs.tg,'lon_around_tg':np.arange(0,int(grid_size/cmip6_resolution))})
lats_da = xr.DataArray(lat_ranges,dims=['tg','lat_around_tg'],coords={'tg':mlr_coefs.tg,'lat_around_tg':np.arange(0,int(grid_size/cmip6_resolution))})

for k,ds in tqdm(ddict.items()): #for each run 
    output_fn = os.path.join(output_dir,'surge',ds.source_id,k.replace(var1,'surge'))
    
    try:
        if (~overwrite_existing) & (output_fn.replace('gs://','') in fs.ls(os.path.join(output_dir,'surge',ds.source_id))):
            continue
    except:
        pass
        
    attrs = ds.attrs
    ds['sfcWind_sqd'] = ds['sfcWind']**2 #add wind squared
    ds['sfcWind_cbd'] = ds['sfcWind']**3 #add wind cubed

    ds = (ds-ds.mean(dim='time'))/ds.std(dim='time',ddof=0) #normalize predictor variables
   
    ds = ds.isel(member_id=0).load() #load into memory
    ds = ds.sel(latitude=lats_da,longitude=lons_da)
    
    #concatenate & stack normalized forcing variables to data array with shape (time,(4 variables * grid_size * grid_size))
    ds['predictors'] = ds[["psl", "sfcWind", "sfcWind_sqd","sfcWind_cbd"]].to_array(dim="predictor_var") 
    ds['predictors'] = ds['predictors'].transpose("time","predictor_var","lon_around_tg",...).stack(f=['predictor_var','lon_around_tg','lat_around_tg'],create_index=False)

    #compute surges from predictors
    surge_ds = xr.Dataset(data_vars=dict(surge=(['time','tg'], np.nan*np.zeros( (len(ds.time),len(mlr_coefs.tg))) )),
                            coords=dict(time=ds.time,tg=mlr_coefs.tg)) #initialize output dataset per model
    
    for t,tg in enumerate(mlr_coefs.tg): #loop over tide gauges (necessary because of EOF analysis?)
        predictors_at_tg = ds.sel(tg=tg)
        mlr_coefs_at_tg = mlr_coefs.mlr_coefs.sel(tg=tg)

        num_pcs = int(np.sum(np.isfinite(mlr_coefs_at_tg)))-1 #number of mlr coefs = number of PCs to derive, intercept doesn't count
        idx_timesteps_w_data = np.argwhere((np.isfinite(predictors_at_tg.predictors).all(axis=1)).values).flatten() #omit timesteps with NaN if any

        #get principal components (using sklearn to keep deterministic signs consistent)
        pca = PCA(num_pcs)
        pca.fit(predictors_at_tg.predictors.isel(time=idx_timesteps_w_data)) #remove missing values for PCA
        pcs = pca.transform(predictors_at_tg.predictors.isel(time=idx_timesteps_w_data))

        components = xr.DataArray(data=pca.components_,dims=['pc','f'],coords=dict(pc=np.arange(num_pcs),f=predictors_at_tg.f))

        #compute RMSEs with ERA5 principal components, only considering the pressure part of the forcing (first num_grid_cells**2)
        rmses = np.sqrt(((components.isel(f=np.arange(num_grid_cells**2))-era5_pcs.sel(tg=tg).isel(f=np.arange(num_grid_cells**2)).isel(pc=np.arange(num_pcs)).component)**2).mean(dim='f')) #original sign
        rmses_flipped = np.sqrt(((components.isel(f=np.arange(num_grid_cells**2))--era5_pcs.isel(f=np.arange(num_grid_cells**2)).sel(tg=tg).isel(pc=np.arange(num_pcs)).component)**2).mean(dim='f')) #opposite sign

        s = (rmses<rmses_flipped).astype('int') #flip pcs if rmse of flipped pc is lower
        s[s==0]=-1
        pcs = pcs * s.values

        #multiply with ERA5 regression coefficients to compute surges
        surge_ds['surge'][idx_timesteps_w_data,t] = np.sum(mlr_coefs_at_tg[np.isfinite(mlr_coefs_at_tg)].values * np.column_stack((np.ones(pcs.shape[0]),pcs)),axis=1) 
    
    surge_ds.attrs        = attrs
    surge_ds              = surge_ds.expand_dims('member_id')
    surge_ds['tg']        = surge_ds.tg.astype('str') #something wrong with encoding object types in zarr, this is the work-around
    surge_ds['member_id'] = surge_ds.member_id.astype('str')
    
    #storage
    surge_ds.chunk({'time':len(surge_ds.time)}).to_zarr('gs://'+output_fn,mode='w')
    surge_ds = {}


  0%|          | 0/138 [00:00<?, ?it/s]